In [ ]:
import numpy as np
import string
from nltk.corpus import stopwords
import csv
import pandas as pd

In [ ]:
url = "./datasets/pre_processed_dataset.csv"
names = ["tweet", "label"]
data = pd.read_csv(url,names=names)
data = data.drop(data.index[0])

In [ ]:
from nltk.tokenize import word_tokenize
import nltk

sentences = data.tweet

stop_words = nltk.corpus.stopwords.words('english')
data_set_tokenized = []

for i in sentences:
    word_tokens = word_tokenize(i)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    data_set_tokenized.append(filtered_sentence)
# filtered_sentence

## Word2Vec with skipgramc

In [ ]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec


In [ ]:

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.get_latest_training_loss = 0
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss() - self.get_latest_training_loss
        self.get_latest_training_loss = model.get_latest_training_loss()
        print('Loss after epoch {}: {}'.format(self.epoch, loss))
        self.epoch += 1


In [ ]:
skip_gram_model = Word2Vec(sentences=data_set_tokenized, vector_size=50, window=2, min_count=4, workers=2, epochs=200, sg=1,
        compute_loss=True, callbacks=[callback()])

In [ ]:
skip_gram_model.wv.most_similar('gmo',topn=10)

In [ ]:
skip_gram_model.wv.most_similar('gmo',topn=10)

In [ ]:
skip_gram_model.wv.most_similar('corn',topn=10)

## Word2Vec with CBOW

In [ ]:
cbow_model = Word2Vec(sentences=data_set_tokenized, vector_size=100, window=10, min_count=4, workers=2, epochs=5, sg=0,
        compute_loss=True, callbacks=[callback()])

In [ ]:
cbow_model.wv.most_similar('crop',topn=10)

## LSTM